# Bibliotecas

In [1]:
#Importar Bibliotecas

import pandas as pd
import numpy as np
import re


import warnings
warnings.filterwarnings("ignore")

 # Documentación

In [2]:
# Importar archivos
df_health = pd.read_csv('../data/HNP_StatsData Alfonso-Copy1.csv')
df_seaice = pd.read_csv('../data/seaice.csv')
df_temp = pd.read_csv('../data/GlobalLandTemperaturesByCountry.csv')
df_temp_change = pd.read_csv('../data/Temperatura_cambio.csv')
df_fires = pd.read_csv('../data/fires.csv')
df_iso = pd.read_csv('../data/iso.csv')
df_agri = pd.read_csv('../data/Agricultura.csv')
df_emissions_origen = pd.read_csv('../data/Emisiones_origen.csv')
df_emisiones = pd.read_csv('../data/owid-co2-data-Copy1.csv')

# Limpieza df_health

In [4]:
#selección de todas las filas que contengan los indicadores que me interesan

df_population = df_health[df_health['Indicator Name'].isin(['Urban population (% of total population)','Urban population growth (annual %)','Rural population (% of total population)', 'Rural population growth (annual %)', 'Population growth (annual %)', 'Population, total'])]
df_health_indicators = df_health[df_health['Indicator Name'].isin(['Treatment for hypertension (% of adults ages 30-79 with hypertension)',\
                                                                   'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',\
                                                                   'Suicide mortality rate (per 100,000 population)','People using at least basic drinking water services (% of population)', \
                                                                   'People using safely managed sanitation services (% of population)',\
                                                                   'Number of surgical procedures (per 100,000 population)',\
                                                                   'Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)',\
                                                                   'Mortality rate attributed to household and ambient air pollution (per 100,000 population)',\
                                                                   'Human capital index (HCI) (scale 0-1)', 'External health expenditure (% of current health expenditure)',\
                                                                   'Current health expenditure (% of GDP)', 'Current health expenditure per capita (current US$)',\
                                                                   'Domestic general government health expenditure (% of GDP)', 'Prevalence of overweight (% of adults)'])]


In [5]:
#Eliminar columnas que no me interesan
df_population = df_population.drop(['Unnamed: 66','Indicator Code'], axis = 1)
df_health_indicators = df_health_indicators.drop(['Unnamed: 66','Indicator Code'], axis = 1)

In [6]:
#Reorganizar los datos dejando 'Year' como columna
df_population = df_population.melt(id_vars = ['Country Name', 'Country Code', 'Indicator Name']).rename({'variable': 'Year'}, axis=1 )
df_health_indicators = df_health_indicators.melt(id_vars = ['Country Name', 'Country Code', 'Indicator Name']).rename({'variable': 'Year'}, axis=1 )